# *Official* Analysis of Cell Selectivity for Elevated Plus Maze (EPM).

#### *Note: Do not add "experimental" code/analysis to this notebook.*

In [1]:
import os
import pandas as pd
import networkx as nx
import numpy as np
import seaborn as sns
import SigProc
from analysis.graph_analysis_utils import NeuronNetwork
from analysis.analysis_utils import FeatureExtractor
from scipy import stats
from IPython.core.interactiveshell import InteractiveShell

In [2]:
%matplotlib inline
InteractiveShell.ast_node_interactivity = "all"

In [3]:
mouse_directory = os.path.join(os.path.expanduser("~"), "Hen_Lab/Mice/EPM")

if not os.path.exists(mouse_directory):
    print("The mouse directory does not exist", file=sys.stderr)

file_num = 0
raw_files = list()
for dir_name, subdir_list, file_list in os.walk(mouse_directory):
    for file_name in file_list:
        if file_name.endswith(".csv"):
            print("{}. full path of: {} is: {}".format(file_num, file_name, os.path.join(dir_name, file_name)))
            file_num += 1
            raw_files.append(os.path.join(dir_name, file_name))

0. full path of: behavior_drd87.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd87/behavior_drd87.csv
1. full path of: Raw_EPM_drd87.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd87/Raw_EPM_drd87.csv
2. full path of: Raw_EPM2_drd87.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd87/Raw_EPM2_drd87.csv
3. full path of: Raw_EPM_drd73.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd73/Raw_EPM_drd73.csv
4. full path of: behavior_drd73.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd73/behavior_drd73.csv
5. full path of: Raw_EPM_drd77.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd77/Raw_EPM_drd77.csv
6. full path of: behavior_drd77.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd77/behavior_drd77.csv
7. full path of: behavior_drd46.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd46/behavior_drd46.csv
8. full path of: Raw_EPM_drd46.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd46/Raw_EPM_drd46.csv


## Import the csv of raw neuron signal data into a pandas DataFrame, and run the data through the Dombeck 2007 signal rocessing algorithm.

##### Note: Recall that the raw neuron signal data for DRD77 and DRD73 must be downsampled by $2$.

In [4]:
drd87_data = pd.read_csv(raw_files[2], header=None)
_, auc_dataframe, cell_transients_dataframe = SigProc.detect_ca_transients_mossy(drd87_data, 2, 0.5, 0.2, 10)

beh_cols = [
    "Trial_time", "Recording_time", "X_center", "Y_center", "Area", "Areachange", "Elongation", "Distance_moved", "Velocity", "Arena_centerpoint", 
    "Open1_centerpoint", "Open2_centerpoint", "Closed1_centerpoint", "Closed2_centerpoint", "OpenArms_centerpoint", "ClosedArms_centerpoint", "Result_1"
]

behavior_df = pd.read_csv(raw_files[0], header=None)
drd87_fe = FeatureExtractor(cell_transients_df=cell_transients_dataframe, auc_df=auc_dataframe, behavior_df=behavior_df, behavior_col_names=beh_cols)

/Users/saveliyyusufov/Hen_Lab/analysis/analysis_utils.py:26: Warning:

Row multiple to downsample behavior dataframe not specified. Behavior dataframe will be downsampled by a row multiple of 3



## We compute the actual $\hat{D}$ values for all of the neuron column vectors in DRD87's `neuron_concated_behavior` dataframe

In [5]:
drd87_real_diff_vals = drd87_fe.set_real_diff_df(drd87_fe.neuron_concated_behavior, drd87_fe.auc_df.columns, "OpenArms_centerpoint", "ClosedArms_centerpoint")
drd87_real_diff_vals

/Users/saveliyyusufov/Hen_Lab/analysis/analysis_utils.py:106: Warning:

Frame rate wasn't specified, so frame rate will be set to 10



,neuron1,neuron2,neuron3,neuron4,neuron5,neuron6,neuron7,neuron8,neuron9,neuron10,...,neuron61,neuron62,neuron63,neuron64,neuron65,neuron66,neuron67,neuron68,neuron69,neuron70
D,5.72857,-3.00185,-1.36973,12.1606,-0.555661,-3.64355,8.0656,-3.7702,-0.683886,-3.68069,...,3.74837,-0.92522,0.176564,0.569157,-1.74818,0.711728,1.67555,1.55181,1.89085,4.47428


## Now, we construct the permutation distributions of the $\hat{D}$ values, for each neuron column vector, from its values in a large number of resamples.
#### Note: use $1000$ resamples for preliminary analysis, and $\ge 10000$ for presentation/publication.

In [6]:
resampled_df = Resampler.shuffle(10000, drd87_fe.neuron_concated_behavior, drd87_fe.auc_df.columns, "OpenArms_centerpoint", "ClosedArms_centerpoint")

NameError: name 'Resampler' is not defined

## Next, we plot all the distribution of each neuron, after permutation resampling.

In [ ]:
resampled_df.hist(alpha=0.5, color='hotpink', bins=50, figsize=(70, 50));

## We can see that almost all of the resampled (permutation) distributions are Gamma distributions. As a result, we must resort to a *non-parametric* test, i.e. the permutation test.

#### **Note**: 

> #### The $t$ test gives accurate $P$-values if the sampling distribution of the difference of means is at least roughly Normal. The permutation test gives accurate $P$-values even when the sampling distribution is not close to Normal. [16-45](http://www2.hawaii.edu/~taylor/z631/ips7e_ch16.pdf)

> #### The permutation test is useful even if we plan to use the two-sample $t$ test. Rather than relying on Normal quantile plots of the two samples and the central limit theorem, we can directly check the Normality of the sampling distribution by looking at the permutation distribution. Permutation tests provide a “gold standard” for assessing two-sample $t$ tests. If the two $P$-values differ considerably, it usually indicates that the conditions for the two-sample $t$ don’t hold for these data. Because permutation tests give accurate $P$-values even when the sampling distribution is skewed, they are often used when accuracy is very important. [16-45](http://www2.hawaii.edu/~taylor/z631/ips7e_ch16.pdf)

> #### When are permutation tests valid? The two-sample t test starts from the condition that the sampling distribution of $\bar{x}_1 − \bar{x}_2$ is Normal. This is the case if both populations have Normal distributions, and it is approximately true for large samples from non-Normal populations because of the central limit theorem. The central limit theorem helps explain the robustness of the two-sample $t$ test. The test works well when both populations are symmetric, especially when the two sample sizes are similar. [16-47](http://www2.hawaii.edu/~taylor/z631/ips7e_ch16.pdf)

> #### The permutation test completely removes the Normality condition. *However, resampling in a way that moves observations between the two groups requires that the two populations are identical when the null hypothesis is true -- not only are their means the same, but also their spreads and shapes.* [16-47](http://www2.hawaii.edu/~taylor/z631/ips7e_ch16.pdf)

### In the case of DRD87, in the EPM experiment), there are $4275$ observations in `ClosedArms_centerpoint` and $605$ observations in `OpenArms_centerpoint`. 
### We are interested in the following test statistic: $$\hat{D} = \texttt{mean}(X_O)− \texttt{mean}(X_C)$$, where $X_O$ are neuron signal values observed in `OpenArms_centerpoint`, and $X_C$ are are neuron signal values observed in `ClosedArms_centerpoint`.

### This gives us $\binom{4880}{605} = \frac{4880!}{605!4275!} \approx 10^{792}$ possible two-sample datasets (under the null hypothesis).

### **How rare is our result?**
> ### Namely, how many of these datasets have $|\texttt{mean}(X_O)− \texttt{mean}(X_C)| \ge \hat{D}$? What is the probability of seeing differences in the group means as extreme (or more extreme) than ours?

### We could consider each of the $\approx 10^{792}$ possible two-sample datasets (note that there are $\approx 10^{80}$ atoms in the observable universe), compute $|\texttt{mean}(X_O)− \texttt{mean}(X_C)|$ for each, and then compare to our original $\hat{D}$. Instead, we use a **permutation test**, which randomly samples from the $10^{792}$ possible two-sample datasets and estimates a $p$-value based on our original $\hat{D}$.

### **Permutation Principle**:
> ### If there were no difference in neuron activity in the OpenArms versus ClosedArms (null hypothesis), then all datasets obtained by randomly assigning the neuron signal data would have equal chance of being observed in the study.

### **Permutation test algorithm**:
> ###  For step $i = 1,2, \cdots ,p$:
> ### (1) Randomly assign neuron signal data points, while holding the behavior data points constant.
> ### (2) Compute $\hat{D}_i = \texttt{mean}(X_O,i)− \texttt{mean}(X_C,i)$, where $X_O,i$ are neuron signal data points in the `OpenArms_centerpoint` behavior in step $i$, and $X_C,i$ are neuron signal data points in the `ClosedArms_centerpoint` behavior in step $i$. 
### Finally, we utilize the function, `compute_two_side_p_val` to compute a (two-sided) $p$-value as follows: $$\frac{1}{p} \sum_{i=1}^{p} \mathbb{1}(|\hat{D}_i| \ge |\hat{D}|)$$

### **What are we actually testing?**
> ### Using the permutation test, we test the null hypothesis that the two samples are from the same distribution.
> ### How do we do this? Have we seen enough evidence (in the form of the observed difference between the sample means being large enough) to reject the null hypothesis that the two groups have identical probability distributions?

##### Sources:
- [STAT GU4206/GR5206 Statistical Computing & Introduction to Data Science. Grabriel Young, Columbia University]()

In [ ]:
drd87_classified_neurons = Resampler.non_normal_neuron_classifier(drd87_fe.auc_df, resampled_df, drd87_real_diff_vals, p_value=0.05, threshold=5)

## After classifying the neurons as either selective, not-selective, or unclassified, we perform a "deeper classification". Namely, we go through and classify each *selective* neuron with a $\hat{D} > 0$ as selective for `OpenArms_centerpoint`. Likewise, we go through and classify each *selective* neuron with a $\hat{D} < 0$ as selective for `ClosedArms_centerpoint`.

In [ ]:
def deep_classify(classified_neurons, real_diff_vals):
    deeper_classification = classified_neurons.copy()

    for neuron in deeper_classification:
        if real_diff_vals[neuron].values > 0:
            if deeper_classification[neuron] == "selective":
                deeper_classification[neuron] = "OpenArms_centerpoint"
        elif real_diff_vals[neuron].values < 0:
            if deeper_classification[neuron] == "selective":
                deeper_classification[neuron] = "ClosedArms_centerpoint"
                
    return deeper_classification

In [ ]:
import plotly
import plotly.graph_objs as go

def plot_pie_chart(dict_of_classifications, *labels):
    values = [sum([1 for classification in dict_of_classifications.values() if classification == "OpenArms_centerpoint"])]
    values.append(sum([1 for classification in dict_of_classifications.values() if classification == "ClosedArms_centerpoint"]))
    values.append(sum([1 for classification in dict_of_classifications.values() if classification == "not-selective"]))
    values.append(sum([1 for classification in dict_of_classifications.values() if classification == "unclassified"]))
    
    trace = go.Pie(labels=labels, values=values)
    plotly.offline.iplot([trace])

# Now, we will repeat the aforementioned procedure to classify the cells of DRD77, DRD73, and DRD46.

## We begin with DRD77.

In [ ]:
drd77_data = pd.read_csv(raw_files[4], header=None)
drd77_data = FeatureExtractor.downsample_dataframe(drd77_data, 2)
_, auc_dataframe, cell_transients_dataframe = SigProc.detect_ca_transients_mossy(drd77_data, 2, 0.5, 0.2, 10)

behavior_df = pd.read_csv(raw_files[5], header=None)
drd77_fe = FeatureExtractor(cell_transients_df=cell_transients_dataframe, auc_df=auc_dataframe, behavior_df=behavior_df, behavior_col_names=beh_cols)

In [ ]:
drd77_real_diff_vals = drd77_fe.set_real_diff_df(drd77_fe.neuron_concated_behavior, drd77_fe.auc_df.columns, "OpenArms_centerpoint", "ClosedArms_centerpoint")
drd77_real_diff_vals

In [ ]:
resampled_df = Resampler.shuffle(10000, drd77_fe.neuron_concated_behavior, drd77_fe.auc_df.columns, "OpenArms_centerpoint", "ClosedArms_centerpoint")
resampled_df.hist(alpha=0.5, color='blue', bins=50, figsize=(70, 50));

In [ ]:
drd77_classified_neurons = Resampler.non_normal_neuron_classifier(drd77_fe.auc_df, resampled_df, drd77_real_diff_vals, p_value=0.05, threshold=5)

## Now, DRD73.

In [ ]:
drd73_data = pd.read_csv(raw_files[2], header=None)
drd73_data = FeatureExtractor.downsample_dataframe(drd73_data, 2)
_, auc_dataframe, cell_transients_dataframe = SigProc.detect_ca_transients_mossy(drd73_data, 2, 0.5, 0.2, 10)

behavior_df = pd.read_csv(raw_files[3], header=None)
drd73_fe = FeatureExtractor(cell_transients_df=cell_transients_dataframe, auc_df=auc_dataframe, behavior_df=behavior_df, behavior_col_names=beh_cols)

In [ ]:
drd73_real_diff_vals = drd73_fe.set_real_diff_df(drd73_fe.neuron_concated_behavior, drd73_fe.auc_df.columns, "OpenArms_centerpoint", "ClosedArms_centerpoint")
drd73_real_diff_vals

In [ ]:
resampled_df = Resampler.shuffle(10000, drd73_fe.neuron_concated_behavior, drd73_fe.auc_df.columns, "OpenArms_centerpoint", "ClosedArms_centerpoint")
resampled_df.hist(alpha=0.5, color='green', bins=50, figsize=(70, 50));

In [ ]:
drd73_classified_neurons = Resampler.non_normal_neuron_classifier(drd73_fe.auc_df, resampled_df, drd73_real_diff_vals, p_value=0.05, threshold=5)

## Finally, DRD46.

In [ ]:
drd46_data = pd.read_csv(raw_files[7], header=None)
_, auc_dataframe, cell_transients_dataframe = SigProc.detect_ca_transients_mossy(drd46_data, 2, 0.5, 0.2, 10)

behavior_df = pd.read_csv(raw_files[6], header=None)
drd46_fe = FeatureExtractor(cell_transients_df=cell_transients_dataframe, auc_df=auc_dataframe, behavior_df=behavior_df, behavior_col_names=beh_cols)

In [ ]:
drd46_real_diff_vals = drd46_fe.set_real_diff_df(drd46_fe.neuron_concated_behavior, drd46_fe.auc_df.columns, "OpenArms_centerpoint", "ClosedArms_centerpoint")
drd46_real_diff_vals

In [ ]:
resampled_df = Resampler.shuffle(10000, drd46_fe.neuron_concated_behavior, drd46_fe.auc_df.columns, "OpenArms_centerpoint", "ClosedArms_centerpoint")
resampled_df.hist(alpha=0.5, color='red', bins=50, figsize=(70, 50));

In [ ]:
drd46_classified_neurons = Resampler.non_normal_neuron_classifier(drd46_fe.auc_df, resampled_df, drd46_real_diff_vals, p_value=0.05, threshold=5)

## Finally, we plot pie charts to visualize the cell selectivity breakdown for:

## DRD87

In [ ]:
drd87_cell_dict = deep_classify(drd87_classified_neurons, drd87_real_diff_vals)
plot_pie_chart(drd87_cell_dict, "OpenArms_centerpoint", "ClosedArms_centerpoint", "not-selective", "unclassified")

In [ ]:
drd87_cell_dict

## DRD77

In [ ]:
drd77_cell_dict = deep_classify(drd77_classified_neurons, drd77_real_diff_vals)
plot_pie_chart(drd77_cell_dict, "OpenArms_centerpoint", "ClosedArms_centerpoint", "not-selective", "unclassified")

## DRD73

In [ ]:
drd73_cell_dict = deep_classify(drd73_classified_neurons, drd73_real_diff_vals)
plot_pie_chart(drd73_cell_dict, "OpenArms_centerpoint", "ClosedArms_centerpoint", "not-selective", "unclassified")

## DRD46

In [ ]:
drd46_cell_dict = deep_classify(drd46_classified_neurons, drd46_real_diff_vals)
plot_pie_chart(drd46_cell_dict, "OpenArms_centerpoint", "ClosedArms_centerpoint", "not-selective", "unclassified")

## Now we concatenate all of our animals' cell selecitivty dictionaries into one dictionary to easily plot the pie chart for all $137$ neurons.

In [ ]:
def concat_dictionaries(*dictionaries):
    all_cells_dictionary = dict()
    key = 1
    for dictionary in dictionaries:
        for value in dictionary.values():
            if key not in all_cells_dictionary:
                all_cells_dictionary[key] = value
                key += 1
                
    return all_cells_dictionary

## For DRD87, DRD77, DRD73, & DRD46, as a collective, we have:

In [ ]:
all_cells_dictionary = concat_dictionaries(drd87_cell_dict, drd77_cell_dict, drd73_cell_dict, drd46_cell_dict)
plot_pie_chart(all_cells_dictionary, "OpenArms_centerpoint", "ClosedArms_centerpoint", "not-selective", "unclassified")

In [ ]:
def activity_by_neurons(concated_df, neuron_names, *behaviors, **kwargs):
    """Computes the neuron activity rates for given behaviors
    
    This function computes the rates for a given animal's activity and  
    neuron, given some set of behaviors.

    Args: 
        concated_df: DataFrame
        
            A concatenated pandas DataFrame of the neuron activity and 
            the corresponding behavior, for a given animal.
        
        neuron_names: list
        
            The names of the neurons whose rates are to be computed.
        
        behaviors: 
            
            The behaviors for which to compute the activity rates. 
        
        frame_rate: int, optional
        
            The framerate to multiply the activity rate by, default is 10.

    Returns: 
        activity_df: a pandas DataFrame of the neuron activity rates.
    """
    frame_rate = kwargs.get("frame_rate", None)
    if frame_rate is None:
        warnings.warn("You did not specify a frame rate, so a frame rate"
                      + " of 10 will be utilized in the computation", Warning)
        frame_rate = 10
        
    activity_df = pd.DataFrame(columns=behaviors)
    
    for behavior in behaviors:
        if behavior in concated_df.columns:
            activity_df.loc[:, behavior] = frame_rate * concated_df.loc[concated_df[behavior] != 0, neuron_names].mean()
        elif '&' in behavior:
            beh1 = behavior.split('&')[0]
            beh2 = behavior.split('&')[1]
            activity_df.loc[:, behavior] = frame_rate * concated_df.loc[(concated_df[beh1] != 0) & ((concated_df[beh2] != 0)), neuron_names].mean()
        elif '|' in behavior:
            beh1 = behavior.split('|')[0]
            beh2 = behavior.split('|')[1]
            activity_df.loc[:, behavior] = frame_rate * concated_df.loc[(concated_df[beh1] != 0) | ((concated_df[beh2] != 0)), neuron_names].mean()

    return activity_df

In [ ]:
import plotly
import plotly.graph_objs as go

def plot_activity_rates(activity_rates_df, **kwargs):
    """Wrapper function for Plotly Bar Charts
        
        This function wraps the Plotly library Bar Chart plotting functionality
        in order to quickly and easily create a bar plot for 
    
    Args:
        activity_rates_df: DataFrame
        
            The pandas DataFrame with all the neuron activity means,
            for corresponding behaviors
            
        title: str, optional
        
            The title of the bar plot, to be displayed above the plot,
            default is "Title Goes Here".
        
        color: str, optional
            
            The color of the bar plot to be plotted, e.g., "rgb(10,10,10)",
            default is "rgb(10,10,200)".
    """
    x = [beh for beh in activity_rates_df.columns]
    y = [activity_rates_df[behavior].mean() for behavior in activity_rates_df.columns]

    title = kwargs.get("title", "Title Goes Here")
    color = kwargs.get("color", "rgb(10,10,200)")
    trace1 = go.Bar(
        x=x,
        y=y,
        text=y,
        marker=dict(
            color=color
        ),
        opacity=0.7
    )

    data = [trace1]
    layout = go.Layout(
        title=title,
    )
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

In [ ]:
def get_continuous_beh_dfs(feature_extractor, behavior, **kwargs):
    seconds = kwargs.get("seconds", 5)
    framerate = kwargs.get("framerate", 10)
    dataframes = list()
    
    continuous_beh = dict()
    for row in feature_extractor.neuron_concated_behavior.itertuples():
        if getattr(row, behavior) == 1:
            continuous_beh[row[0]] = 1
        else:
            if len(continuous_beh) >= seconds*framerate:

                # Note the start frame, and the end frame for when the animal entered the area.
                begin = list(continuous_beh.keys())[0]
                end = list(continuous_beh.keys())[len(list(continuous_beh.keys()))-1]

                dataframes.append(feature_extractor.neuron_concated_behavior.iloc[begin:end])

                continuous_beh.clear()
            else:
                continuous_beh.clear()
                
    return dataframes

In [ ]:
def get_rates_for_cont_beh(feature_extractor, behavior, seconds):
    dataframes = get_continuous_beh_dfs(feature_extractor, behavior, seconds=seconds)
    
    rates = list()
    for df in dataframes:
        activity_rates_df = activity_by_neurons(df, feature_extractor.auc_df.columns, behavior)
        rates.append(activity_rates_df[behavior].mean())
        
    return rates, dataframes

In [ ]:
def sort_rates_chronological(feature_extractor, open_arms_dataframes, closed_arms_dataframes):
    i = 0
    j = 0
    rates = list()
    markers_on = list()

    while i < len(open_arms_dataframes):
        if open_arms_dataframes[i].index.values[-1] < closed_arms_dataframes[j].index.values[-1]:
            # print("Appended OpenArms dataframe with indices: {}:{}".format(open_arms_dataframes[i].index.values[0], open_arms_dataframes[i].index.values[-1]))
            activity_rates_df = activity_by_neurons(open_arms_dataframes[i], feature_extractor.auc_df.columns, "OpenArms_centerpoint")
            rates.append(activity_rates_df["OpenArms_centerpoint"].mean())
            markers_on.append(len(rates)-1)
            i += 1
        else:
            # print("Appended ClosedArms dataframe with indices: {}:{}".format(closed_arms_dataframes[j].index.values[0], closed_arms_dataframes[j].index.values[-1]))
            activity_rates_df = activity_by_neurons(closed_arms_dataframes[j], feature_extractor.auc_df.columns, "ClosedArms_centerpoint")
            rates.append(activity_rates_df["ClosedArms_centerpoint"].mean())
            j += 1

    while j < len(closed_arms_dataframes):
        # print("Appended ClosedArms dataframe with indices: {}:{}".format(closed_arms_dataframes[j].index.values[0], closed_arms_dataframes[j].index.values[-1]))
        activity_rates_df = activity_by_neurons(closed_arms_dataframes[j], feature_extractor.auc_df.columns, "ClosedArms_centerpoint")
        rates.append(activity_rates_df["ClosedArms_centerpoint"].mean())
        j += 1
    
    return rates, markers_on

In [ ]:
import matplotlib.pyplot as plt

def quick_plot(y_axis, figsize, color="blue", title="Title Goes Here", markevery=None):
    plt.figure(figsize=figsize)
    plt.plot(y_axis, color=color, marker='o', linestyle="solid", linewidth=1, markersize=7, markevery=markevery)
    plt.xlabel("Entry")
    plt.ylabel("Rate")
    plt.title(title)
    plt.grid(True)
    plt.show();

## DRD87

In [ ]:
drd87_open_arms_rates, drd87_open_arms_dataframes = get_rates_for_cont_beh(drd87_fe, "OpenArms_centerpoint", 2)
drd87_closed_arms_rates, drd87_closed_arms_dataframes = get_rates_for_cont_beh(drd87_fe, "ClosedArms_centerpoint", 2)

quick_plot(drd87_open_arms_rates, (10, 5), color="red", title="DRD87 Activity Rate per Entry into OpenArms")
quick_plot(drd87_closed_arms_rates, (10, 5), color="blue", title="DRD87 Activity Rate per Entry into ClosedArms")

drd87_all_rates, drd87_markers_on = sort_rates_chronological(drd87_fe, drd87_open_arms_dataframes, drd87_closed_arms_dataframes)

quick_plot(drd87_all_rates, (10, 5), color="green", title="DRD87 Activity Rate per Entry", markevery=drd87_markers_on)

## DRD77

In [ ]:
drd77_open_arms_rates, drd77_open_arms_dataframes = get_rates_for_cont_beh(drd77_fe, "OpenArms_centerpoint", 2)
drd77_closed_arms_rates, drd77_closed_arms_dataframes = get_rates_for_cont_beh(drd77_fe, "ClosedArms_centerpoint", 2)

quick_plot(drd77_open_arms_rates, (10, 5), color="red", title="DRD77 Activity Rate per Entry into OpenArms")
quick_plot(drd77_closed_arms_rates, (10, 5), color="blue", title="DRD77 Activity Rate per Entry into ClosedArms")

drd77_all_rates, drd77_markers_on = sort_rates_chronological(drd77_fe, drd77_open_arms_dataframes, drd77_closed_arms_dataframes)

quick_plot(drd77_all_rates, (10, 5), color="green", title="DRD77 Activity Rate per Entry", markevery=drd77_markers_on)

## DRD73

In [ ]:
drd73_open_arms_rates, drd73_open_arms_dataframes = get_rates_for_cont_beh(drd73_fe, "OpenArms_centerpoint", 2)
drd73_closed_arms_rates, drd73_closed_arms_dataframes = get_rates_for_cont_beh(drd73_fe, "ClosedArms_centerpoint", 2)

quick_plot(drd73_open_arms_rates, (10, 5), color="red", title="DRD73 Activity Rate per Entry into OpenArms")
quick_plot(drd73_closed_arms_rates, (10, 5), color="blue", title="DRD73 Activity Rate per Entry into ClosedArms")

drd73_all_rates, drd73_markers_on = sort_rates_chronological(drd73_fe, drd73_open_arms_dataframes, drd73_closed_arms_dataframes)

quick_plot(drd73_all_rates, (10, 5), color="green", title="DRD73 Activity Rate per Entry", markevery=drd73_markers_on)

## DRD46

In [ ]:
drd46_open_arms_rates, drd46_open_arms_dataframes = get_rates_for_cont_beh(drd46_fe, "OpenArms_centerpoint", 2)
drd46_closed_arms_rates, drd46_closed_arms_dataframes = get_rates_for_cont_beh(drd46_fe, "ClosedArms_centerpoint", 2)

quick_plot(drd46_open_arms_rates, (10, 5), color="red", title="DRD46 Activity Rate per Entry into OpenArms")
quick_plot(drd46_closed_arms_rates, (10, 5), color="blue", title="DRD46 Activity Rate per Entry into ClosedArms")

drd46_all_rates, drd46_markers_on = sort_rates_chronological(drd46_fe, drd46_open_arms_dataframes, drd46_closed_arms_dataframes)

quick_plot(drd46_all_rates, (10, 5), color="green", title="DRD46 Activity Rate per Entry", markevery=drd46_markers_on)

# Now, we will investigate whether or not *OpenArms_centerpoint* cells are more/same/less likely to also be active during other anxiety tasks, i.e., POPP, OFT.

In [ ]:
DRD87_POPP_RAW = pd.read_csv("~/Hen_Lab/Mice/DRD87_NO_OFT_POPP_reg/D87_POPP_C_raw.csv" ,header=None)
_, auc_dataframe, cell_transients_dataframe = SigProc.detect_ca_transients_mossy(DRD87_POPP_RAW, 2, 0.5, 0.2, 10)

In [ ]:
behavior_df = pd.read_csv("~/Hen_Lab/Mice/DRD87_NO_OFT_POPP_reg/Behavior_DRD87_POPP.csv")
behavior_df.drop(columns=["X center", "Y center", "Area", "Areachange", "Elongation", "Distance moved", "Velocity", "Result 1"], inplace=True)
behavior_df.columns = ["Trial Time", "Recording Time", "Arena_centerpoint", "Odor_centerpoint", "Neutral_centerpoint", "Odor_Cup_centerpoint", "Neutral_Cup_centerpoint"]

In [ ]:
drd87_fe_popp = FeatureExtractor(cell_transients_df=cell_transients_dataframe, auc_df=auc_dataframe, behavior_df=behavior_df, row_multiple=2)

In [ ]:
drd87_popp_activity = activity_by_neurons(drd87_fe_popp.neuron_concated_behavior, drd87_fe_popp.auc_df.columns, "Arena_centerpoint", "Odor_centerpoint", "Neutral_centerpoint", "Odor_Cup_centerpoint", "Neutral_Cup_centerpoint")

In [ ]:
neuron_dictionary = pd.read_csv("~/Hen_Lab/Mice/DRD87_NO_OFT_POPP_reg/EPM_NO_OFT_POPP_cellreg_dict.csv",header=None)
neuron_dictionary.columns = ["EPM", "NO", "OFT", "POPP"]

# Reset the index so that it starts from 1, since we refer to neurons as neurons 1, 2, ..., n
neuron_dictionary.index = pd.RangeIndex(1, len(neuron_dictionary.index)+1)

## Now we create a dictionary to translate neurons from day $1$ (EPM) to day $2$, (NO).

In [ ]:
epm_to_popp_dict = dict()

for row in neuron_dictionary.itertuples():
    epm_neuron = getattr(row, "EPM")
    popp_neuron = getattr(row, "POPP")
    
    if popp_neuron == 0:
        continue
    
    epm_to_popp_dict[epm_neuron] = popp_neuron
    print(epm_neuron, popp_neuron)

In [ ]:
plot_activity_rates(drd87_popp_activity, title="DRD87 Novel Object Activity Rates")

In [ ]:
drd87_real_diff_vals = drd87_fe_popp.set_real_diff_df(drd87_fe_popp.neuron_concated_behavior, drd87_fe_popp.auc_df.columns, "Odor_centerpoint", "Neutral_centerpoint")
drd87_real_diff_vals

In [ ]:
resampled_df = Resampler.shuffle(10000, drd87_fe_popp.neuron_concated_behavior, drd87_fe_popp.auc_df.columns, "Odor_centerpoint", "Neutral_centerpoint")
resampled_df.hist(alpha=0.5, color='blue', bins=50, figsize=(70, 50));

In [ ]:
drd87_classified_neurons = Resampler.non_normal_neuron_classifier(drd87_fe_popp.auc_df, resampled_df, drd87_real_diff_vals, p_value=0.05, threshold=5)

In [ ]:
drd87_classified_neurons